## How to use calculable objectives

In [ ]:
# Import the variable, objectives, sampler, acquisition function, and the optimisation classes
from nemo_bo.opt.variables import ContinuousVariable, VariablesList
from nemo_bo.opt.objectives import RegressionObjective, ObjectivesList
from nemo_bo.acquisition_functions.expected_improvement.expected_improvement import (
    ExpectedImprovement,
)
from nemo_bo.opt.samplers import LatinHyperCubeSampling
from nemo_bo.opt.optimisation import Optimisation

In [ ]:
# Create the variable objects
var1 = ContinuousVariable(name="variable1", lower_bound=0.0, upper_bound=100.0)
var2 = ContinuousVariable(name="variable2", lower_bound=0.0, upper_bound=100.0)
var_list = VariablesList([var1, var2])

### 1. Define the function to use for converting the variable data into the target value
The first step to creating a calculable objective is to define the function to use for converting the variable data into the target value. For example, this can be used for determining the exact cost of a set of variables. This function inherits the `DeterministicFunction` class.

A key aspect of this function is the `obj_function_data` keyword argument that is required when the class is instantiated because it contains the information required for evaluation.

The `evaluate` function is required to return a tuple of the calculated value and the standard deviation of the calculated value. For example, the `Cost_calculation` class shown below has an evaluate function that returns the exact cost and a standard deviation of zero for each cost.

Following this, the function class written is instantiated whilst passing in any required information.

### 2. Specifying an optimisation target as a calculable objective
The `CalculableObjective` class shares many arguments and keyword arguments as the `RegressionObjective` class except that a `DeterministicFunction` is be passed using the `ob_function` argument.

In [ ]:
# Define the deterministic function for the calculable objective
from typing import Dict, Tuple

import numpy as np
from nemo_bo.opt.objectives import DeterministicFunction
from nemo_bo.opt.objectives import CalculableObjective


# Cost calculation used for this example
class Cost_calculation(DeterministicFunction):
    def __init__(self, obj_function_data: Dict):
        super().__init__(obj_function_data)

    def evaluate(self, X: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        cost_variable1 = X[:, 0] * self.obj_function_data["variable1_cost_per_kg"]
        cost_variable2 = X[:, 1] * self.obj_function_data["variable2_cost_per_kg"]

        cost_total = cost_variable1 + cost_variable2
        cost_stddev = np.zeros_like(cost_total)

        return cost_total, cost_stddev


# Instantiate the deterministic function and pass in the any required calculation information
obj_function_data = {"variable1_cost_per_kg": 0.95, "variable2_cost_per_kg": 2.05}
det_func = Cost_calculation(obj_function_data)

# Define the calculable objective
obj1 = CalculableObjective(
    name="Cost",
    obj_max_bool=False,
    lower_bound=0.0,
    upper_bound=7.5,
    obj_function=det_func,
    units="$",
)

In [ ]:
# Create the objective objects to be modelled using a machine learning model
obj2 = RegressionObjective(name="objective2", obj_max_bool=False, lower_bound=0.0, upper_bound=100.0)
obj_list = ObjectivesList([obj1, obj2])

In [ ]:
# Instantiate the sampler
sampler = LatinHyperCubeSampling()

In [ ]:
# Instantiate the acquisition function
acq_func = ExpectedImprovement(num_candidates=4)

In [ ]:
# Set up the optimisation instance
optimisation = Optimisation(var_list, obj_list, acq_func, sampler=sampler)

In [ ]:
# Start the optimisation using the convenient run function that will run for the specified number of iterations
# X and Y arrays represent a hypothetical initial dataset
optimisation_data = optimisation.run(X, Y, number_of_iterations=50)